# Building a Exa (formerly Metaphor) Data Agent

This tutorial walks through using the LLM tools provided by the [Exa API](https://exa.ai) to allow LLMs to easily search and retrieve HTML content from the Internet.

To get started, you will need an [OpenAI api key](https://platform.openai.com/account/api-keys) and an [Exa API key](https://dashboard.exa.ai/overview)

We will import the relevant agents and tools and pass them our keys here:

In [10]:
# Set up OpenAI
import os
import openai
from llama_index.agent import OpenAIAgent

openai.api_key = os.environ["OPENAI_API_KEY"]

# Set up Metaphor tool
from llama_hub.tools.exa.base import ExaToolSpec

exa_tool = ExaToolSpec(
    api_key=os.environ["EXA_API_KEY"],
)

exa_tool_list = exa_tool.to_tool_list()
for tool in exa_tool_list:
    print(tool.metadata.name)

search
retrieve_documents
search_and_retrieve_documents
find_similar
current_date


## Testing the Exa tools

We've imported our OpenAI agent, set up the api key, and initialized our tool, checking the methods that it has available. Let's test out the tool before setting up our Agent.

All of the Exa search tools make use of the `AutoPrompt` option where Exa will pass the query through an LLM to refine and improve it.

In [12]:
exa_tool.search_and_retrieve_documents("machine learning transformers", num_results=3)

ValueError: Invalid option: 'max_length'

In [4]:
exa_tool.find_similar(
    "https://www.mihaileric.com/posts/transformers-attention-in-disguise/"
)

[{'title': 'Transformers: Attention in Disguise',
  'url': 'https://www.mihaileric.com/posts/transformers-attention-in-disguise/',
  'id': 'iEYMai5rS9k0hN5_BH0VZg'},
 {'title': 'Transformers: a Primer',
  'url': 'http://www.columbia.edu/~jsl2239/transformers.html',
  'id': 'cXbT9IsR5u8NtLTAIcUWOA'},
 {'title': 'Illustrated Guide to Transformers- Step by Step Explanation',
  'url': 'https://towardsdatascience.com/illustrated-guide-to-transformers-step-by-step-explanation-f74876522bc0?gi=8fe76db5c4d9',
  'id': 'czTCPZ1vqo-f92WQwKvRig'}]

In [4]:
exa_tool.search_and_retrieve_documents(
    "This is the best explanation for machine learning transformers:", num_results=1
)

ValueError: Invalid option: 'max_length'

We can see we have different tools to search for results, retrieve the results, find similar results to a web page, and finally a tool that combines search and document retrieval into a single tool. We will test them out in LLM Agents below:

### Using the Search and Retrieve documents tools in an Agent

We can create an agent with access to the above tools and start testing it out:

In [4]:
# We don't give the Agent our unwrapped retrieve document tools, instead passing the wrapped tools
agent = OpenAIAgent.from_tools(
    exa_tool_list,
    verbose=True,
)

In [5]:
print(agent.chat("What are the best resturants in toronto?"))

=== Calling Function ===
Calling function: search with args: {
  "query": "best restaurants in Toronto"
}
[Metaphor Tool] Autoprompt: Here's a great restaurant in Toronto:
Got output: [{'title': 'Via Allegro Ristorante - Toronto Fine Dining Restaurant', 'url': 'https://viaallegroristorante.com/', 'id': 'EVlexzJh-lzkVr4tb2y_qw'}, {'title': 'Location', 'url': 'https://osteriagiulia.ca/', 'id': 'HjP5c54vqb3n3UNa3HevSA'}, {'title': 'PATOIS • TORONTO', 'url': 'https://www.patoistoronto.com/', 'id': 'h3A1DO4glTdK2yw0XV_QKQ'}, {'title': 'Sophisticated Dining - Toronto Restaurant | Scaramouche', 'url': 'https://www.scaramoucherestaurant.com/', 'id': 'QWkwiSzB3JwT7gUgg2mhig'}, {'title': 'The Carbon Bar & Restaurant', 'url': 'https://thecarbonbar.ca/', 'id': 'Ekud2zFGBZONuDkxcBMF7w'}, {'title': "Serving Yorkville in Toronto since 1967 - Joso's Restaurant", 'url': 'http://www.josos.com/', 'id': 'OTgYucyLGJh64yVTzGHDpA'}, {'title': 'La Fenice', 'url': 'https://www.lafenice.ca/', 'id': 'M-LHQZP6V40

In [6]:
print(agent.chat("tell me more about Osteria Giulia"))

=== Calling Function ===
Calling function: find_similar with args: {
  "url": "https://osteriagiulia.ca/",
  "num_results": 1
}
Got output: [{'title': 'Location', 'url': 'https://osteriagiulia.ca/', 'id': 'HjP5c54vqb3n3UNa3HevSA'}]
=== Calling Function ===
Calling function: retrieve_documents with args: {
  "ids": ["HjP5c54vqb3n3UNa3HevSA"]
}
Got output: [Document(id_='7bd8327a-34cc-46ce-b731-1e70bb2f2ce6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='f5d55bc509de58dc2ac7bf91e54d8054c59df2b20337e73d78859256736f393f', text='<div><div><li><div>\n<div><div><p></p><h3>Location</h3><p></p></div><div><p></p><h3>134 AVENUE ROAD</h3><p></p></div><div><p></p><h3>Toronto, ON</h3><p></p></div><div><p></p><h2><a href="tel:416.964.8686">416.964.8686</a></h2><p></p></div><div><p></p><h2><a href="mailto:info@osteriagiulia.ca">info@osteriagiulia.ca</a></h2><p></p></div><div><p></p><h2>(for general inquires only...<br />\nno reserv

## Avoiding Context Window Issues

The above example shows the core uses of the Exa tool. We can easily retrieve a clean list of links related to a query, and then we can fetch the content of the article as a cleaned up html extract. Alternatively, the search_and_retrieve_documents tool directly returns the documents from our search result.

We can see that the content of the articles is somewhat long compared to current LLM context windows, and so to allow retrieval and summary of many documents we will set up and use another tool from LlamaIndex that allows us to load text into a VectorStore, and query it for retrieval. This is where the `search_and_retrieve_documents` tool become particularly useful. The Agent can make a single query to retrieve a large number of documents, using a very small number of tokens, and then make queries to retrieve specific information from the documents.

In [7]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

# The search_and_retrieve_documents tool is the third in the tool list, as seen above
wrapped_retrieve = LoadAndSearchToolSpec.from_defaults(
    exa_tool_list[2],
)

Our wrapped retrieval tools separate loading and reading into separate interfaces. We use `load` to load the documents into the vector store, and `read` to query the vector store. Let's try it out again

In [8]:
wrapped_retrieve.load("This is the best explanation for machine learning transformers:")
print(wrapped_retrieve.read("what is a transformer"))
print(wrapped_retrieve.read("who wrote the first paper on transformers"))

[Metaphor Tool] Autoprompt: Here is a great article explaining machine learning transformers:

A transformer is a type of artificial neural network used in natural language processing (NLP) tasks such as machine translation, text summarization, and question answering. It is based on the Transformer architecture, which was introduced in 2017 by Google researchers. The Transformer architecture uses attention mechanisms to learn the relationships between words in a sentence, allowing it to better understand the context of the sentence and generate more accurate translations.

The first paper on transformers was written by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin in 2017. The paper was titled "Attention Is All You Need".


## Creating the Agent

We now are ready to create an Agent that can use Metaphors services to it's full potential. We will use our wrapped read and load tools, as well as the `get_date` utility for the following agent and test it out below:

In [12]:
# Just pass the wrapped tools and the get_date utility
agent = OpenAIAgent.from_tools(
    [*wrapped_retrieve.to_tool_list(), exa_tool_list[4]],
    verbose=True,
)

In [13]:
print(
    agent.chat(
        "Can you summarize everything published in the last month regarding news on"
        " superconductors"
    )
)

=== Calling Function ===
Calling function: current_date with args: {}
Got output: 2023-08-20
=== Calling Function ===
Calling function: search_and_retrieve_documents with args: {
  "query": "superconductors",
  "start_published_date": "2023-07-20",
  "end_published_date": "2023-08-20"
}
[Metaphor Tool] Autoprompt: "Here is an interesting article about superconductors:
Got output: Content loaded! You can now search the information using read_search_and_retrieve_documents
=== Calling Function ===
Calling function: read_search_and_retrieve_documents with args: {
  "query": "superconductors"
}
Got output: 
Superconductors are materials that can perfectly conduct electricity. They are used in a variety of applications, such as particle accelerators, nuclear fusion devices, MRI machines, and maglev trains. However, so far, no superconductor has been proven to work at ambient pressures and temperatures. On July 22, scientists in South Korea published research claiming to have solved this prob

We asked the agent to retrieve documents related to superconductors from this month. It used the `get_date` tool to determine the current month, and then applied the filters in Metaphor based on publication date when calling `search`. It then loaded the documents using `retrieve_documents` and read them using `read_retrieve_documents`.

We can make another query to the vector store to read from it again, now that the articles are loaded: